<a href="https://colab.research.google.com/github/Unfeir/DS/blob/main/HW_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811631 sha256=4b2d0c599f2a5fc27c92637db5a1e92db76e091817b007b3eb3e7a08b517ea5c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
from surprise import Dataset, SVD, SVDpp, NMF, Reader
from surprise.model_selection import cross_validate
from collections import defaultdict
from surprise.model_selection import train_test_split

In [ ]:
# links = pd.read_csv('/content/drive/MyDrive/Data sets/ml-25m/links.csv')
# movies = pd.read_csv('/content/drive/MyDrive/Data sets/ml-25m/movies.csv')
# ratings = pd.read_csv('/content/drive/MyDrive/Data sets/ml-25m/ratings.csv')
# tags = pd.read_csv('/content/drive/MyDrive/Data sets/ml-25m/tags.csv')
# ratings, movies, ratings, tags

In [3]:
data = Dataset.load_builtin("ml-100k")


Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [19]:
raw_data = data.raw_ratings


for i in range(5):
    print(raw_data[i])

('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [6]:
algos = [SVD(),  NMF(), SVDpp()]
for algo in algos:
    results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    print(f"Algorithm: {algo.__class__.__name__}")
    print("RMSE:", results['test_rmse'].mean())
    print("MAE:", results['test_mae'].mean())

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9374  0.9366  0.9306  0.9472  0.9310  0.9365  0.0060  
MAE (testset)     0.7400  0.7406  0.7338  0.7475  0.7313  0.7386  0.0057  
Fit time          0.99    0.97    0.98    0.97    0.96    0.97    0.01    
Test time         0.13    0.10    0.16    0.09    0.16    0.13    0.03    
Algorithm: SVD
RMSE: 0.9365473020225391
MAE: 0.738643689510909
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9612  0.9671  0.9629  0.9660  0.9632  0.9641  0.0021  
MAE (testset)     0.7597  0.7583  0.7530  0.7589  0.7587  0.7577  0.0024  
Fit time          1.73    2.12    1.73    1.62    1.66    1.77    0.18    
Test time         0.14    0.16    0.09    0.16    0.08    0.13    0.04    
Algorithm: NMF
RMSE: 0.9640832071693112
MAE: 0.7577122721334051
Evaluating RMSE

In [37]:
param_grid = {'n_factors': [50, 100, 150],
              'n_epochs': [20, 30, 40],
              'lr_all': [0.005, 0.01, 0.02],
              'reg_all': [0.02, 0.1, 0.4]}

for i in range(3):
  model = SVD(n_factors=param_grid['n_factors'][i], n_epochs=param_grid['n_epochs'][i], lr_all=param_grid['lr_all'][i], reg_all=param_grid['reg_all'][i])
  results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
  print(f"SVD variants waz: n_factors={param_grid['n_factors'][i]} | n_epochs={param_grid['n_epochs'][i]} | lr_all={param_grid['lr_all'][i]} | reg_all={param_grid['reg_all'][i]}")
  print("RMSE:", results['test_rmse'].mean())
  print("MAE:", results['test_mae'].mean())





SVD variants waz: n_factors=50 | n_epochs=20 | lr_all=0.005 | reg_all=0.02
RMSE: 0.9349611468205508
MAE: 0.7367845218639859
SVD variants waz: n_factors=100 | n_epochs=30 | lr_all=0.01 | reg_all=0.1
RMSE: 0.9118542853054455
MAE: 0.7211179379762498
SVD variants waz: n_factors=150 | n_epochs=40 | lr_all=0.02 | reg_all=0.4
RMSE: 0.9581153113868874
MAE: 0.7677565918538136


In [38]:
for n_factors in param_grid['n_factors']:
    for n_epochs in param_grid['n_epochs']:
        for lr_all in param_grid['lr_all']:
            for reg_all in param_grid['reg_all']:
              model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)
              results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
              print(f"RMSE: {results['test_rmse'].mean():.5f} | MAE: {results['test_mae'].mean():.5f} ||| {n_factors=}, {n_epochs=}, {lr_all=}, {reg_all=}")




RMSE: 0.93403 | MAE: 0.73651 ||| n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02
RMSE: 0.94026 | MAE: 0.74544 ||| n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.1
RMSE: 0.95688 | MAE: 0.76640 ||| n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.4
RMSE: 0.95523 | MAE: 0.74820 ||| n_factors=50, n_epochs=20, lr_all=0.01, reg_all=0.02
RMSE: 0.92285 | MAE: 0.73065 ||| n_factors=50, n_epochs=20, lr_all=0.01, reg_all=0.1
RMSE: 0.95526 | MAE: 0.76471 ||| n_factors=50, n_epochs=20, lr_all=0.01, reg_all=0.4
RMSE: 1.00081 | MAE: 0.77971 ||| n_factors=50, n_epochs=20, lr_all=0.02, reg_all=0.02
RMSE: 0.91275 | MAE: 0.72136 ||| n_factors=50, n_epochs=20, lr_all=0.02, reg_all=0.1
RMSE: 0.95903 | MAE: 0.76860 ||| n_factors=50, n_epochs=20, lr_all=0.02, reg_all=0.4
RMSE: 0.93897 | MAE: 0.73726 ||| n_factors=50, n_epochs=30, lr_all=0.005, reg_all=0.02
RMSE: 0.93164 | MAE: 0.73820 ||| n_factors=50, n_epochs=30, lr_all=0.005, reg_all=0.1
RMSE: 0.95544 | MAE: 0.76530 ||| n_factors=50, n_epochs=

Порівняня алгоритмів **SVD,  NMF, SVDpp** Використання SVDpp показало дуже мале покращення: RMSE на 0,016 та MAE на 0,016 При значному зросту витрат часу на навчання. NMF показав найгірший результат

Підбір варіантів  n_factors, n_epoch, lr_all, reg_all для моделі SVD хоч не дав суттєвих змін в результатах, але покращення були кращі ніж при переборі різних моделей.

При SVDpp зі стндартними параметрами результати були:

RMSE: 0.9581 MAE: 0.7677


При SVD з підбором параметрів вдалося покращити до:

RMSE: 0.91009 | MAE: 0.71907
З параметрами  n_factors=150, n_epochs=40, lr_all=0.01, reg_all=0.1

---

